In [1]:
# 패키지 로드
import geopandas as gpd
import pandas as pd
import os

In [2]:
# GDAL_DATA 환경 변수 경로 설정
os.environ['GDAL_DATA'] = r'C:\Users\TG\miniconda3\envs\moonsoo1\Library\share\gdal'

In [20]:
# shp 파일 읽기(한글 속성값 때문에 EUC-KR 인코딩 지정)
data_shp_0 = gpd.read_file('D:/GIS/행정구역/리/동리_유역_병합.shp', encoding='euc-kr')

data_shp_0

,DONGLI,법정동코드,법정동명,시도,시군구,읍면동,리,AREA_TOTAL,SW_NAME,AREA_WATER,RATIO,geometry
0,봉의동,5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,177444.458,북한C,1.774445e+05,100.0,"POLYGON ((263858.449 587613.572, 263859.62 587..."
1,요선동,5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,None,88350.298,북한C,8.835030e+04,100.0,"POLYGON ((263949.193 587360.716, 263974.087 58..."
2,낙원동,5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,107927.810,북한C,1.079278e+05,100.0,"POLYGON ((263564.29 586554.824, 263559.02 5865..."
3,중앙로1가,5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,37470.002,북한C,3.747000e+04,100.0,"POLYGON ((264184.523 587301.17, 264144.988 587..."
4,중앙로2가,5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,44367.327,북한C,4.436733e+04,100.0,"POLYGON ((263722.617 586664.076, 263706.017 58..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1924,None,None,None,None,None,None,None,NaN,소양A,1.165400e+01,NaN,"MULTIPOLYGON (((325914.33 606777.035, 325910.6..."
1925,None,None,None,None,None,None,None,NaN,인북A,1.804567e+08,NaN,"MULTIPOLYGON (((297052.084 655909.91, 297163.9..."
1926,None,None,None,None,None,None,None,NaN,소양B,2.679938e+05,NaN,"MULTIPOLYGON (((302726.162 587979.488, 302791...."
1927,None,None,None,None,None,None,None,NaN,제천A,3.800844e+08,NaN,"MULTIPOLYGON (((295445.848 523282.229, 295443...."


In [26]:
# 'geometry' 열 제거
data_shp = data_shp_0.drop(columns=['DONGLI', '법정동명', 'geometry'])

# 열 이름 변경
data_shp = data_shp.rename(columns={'시군구': '시군', 'AREA_TOTAL': '동리면적', 'SW_NAME': '단위유역', 'AREA_WATER': '유역면적', 'RATIO': '유역면적비율'})

# 동리면적, 유역면적, 유역면적비율 필드의 값이 NAN이거나 0인 행 제거
data_shp = data_shp.dropna(subset=['동리면적', '유역면적', '유역면적비율'])
data_shp = data_shp[(data_shp['동리면적'] != 0) & (data_shp['유역면적'] != 0) & (data_shp['유역면적비율'] != 0)]

# 시군, 읍면동, 동리 열 값을 조합하여 '동리코드' 열 추가
data_shp.insert(
    1,
    '동리코드',
    (
        data_shp['시군'] + ' ' +
        data_shp['읍면동'] + ' ' +
        data_shp['리'].fillna('')
    ).str.strip()
)

# '법정동코드' 열 속성을 'numeric'으로 변경
data_shp['법정동코드'] = pd.to_numeric(data_shp['법정동코드'], errors='coerce')

data_shp

,법정동코드,동리코드,시도,시군,읍면동,리,동리면적,단위유역,유역면적,유역면적비율
0,5111010100,춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0
1,5111010200,춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0
2,5111010300,춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0
3,5111010400,춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0
4,5111010500,춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0
...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,None,2.518901e+06,100.0
1903,5183035039,양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,None,9.178029e+05,100.0
1904,5183035040,양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,None,4.658669e+05,100.0
1905,5183035041,양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,None,1.361706e+06,100.0


In [27]:
# 동리면적에서 유역면적의 비중 계산(백분율 소수점 둘째자리까지)
# data_shp['유역면적비율'] = (data_shp['유역면적'] / data_shp['동리면적']) * 100
# data_shp['유역면적비율'] = data_shp['유역면적비율'].round(2)

# 점유율 열을 추가하고, 유역면적비율 열의 값이 5미만인 경우 0으로 설정, 95 이상인 경우 100으로 설정, 이외의 경우 유역면적비율의 값을 그대로 사용
data_shp['점유율'] = data_shp['유역면적비율'].apply(lambda x: 0 if x < 5 else (100 if x >= 95 else x))

# 동리개수 열을 추가하고, '동리코드'열의 값이 동일한 행의 갯수를 세어서 표시
data_shp['동리개수'] = data_shp.groupby(['동리코드'])['동리코드'].transform('count')

data_shp

,법정동코드,동리코드,시도,시군,읍면동,리,동리면적,단위유역,유역면적,유역면적비율,점유율,동리개수
0,5111010100,춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0,100.0,1
1,5111010200,춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0,100.0,1
2,5111010300,춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0,100.0,1
3,5111010400,춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0,100.0,1
4,5111010500,춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,None,2.518901e+06,100.0,100.0,1
1903,5183035039,양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,None,9.178029e+05,100.0,100.0,1
1904,5183035040,양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,None,4.658669e+05,100.0,100.0,1
1905,5183035041,양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,None,1.361706e+06,100.0,100.0,1


In [28]:
#
def adjust_유역면적비율s(group):
    # 분할개수가 2 이하인 경우는 재조정 없이 그대로 반환
    if group['동리개수'].iloc[0] <= 2:
        return group

    # 분할개수가 3 이상인 경우 재조정 실시
    # 이미 0으로 처리된 값과 100으로 고정된 값은 그대로 유지하고,
    # 나머지(0 < 비율_수정 < 100)에 대해 합이 100이 되도록 재분배합니다.
    fixed_mask = group['점유율'] == 100  # 100인 경우는 고정
    # 0인 경우는 이미 5% 미만 처리되어 0이므로 그대로 둡니다.
    variable_mask = (group['점유율'] > 0) & (group['점유율'] < 100)
    
    # 이미 고정된 비율의 합(여기서는 100이 고정된 값만 고려)
    total_fixed = group.loc[fixed_mask, '점유율'].sum()
    # 가변 비율의 합(0 < 비율_수정 < 100인 값들)
    total_variable = group.loc[variable_mask, '점유율'].sum()
    
    # 만약 재조정할 대상이 있다면, 나머지 비율의 총합이 100-total_fixed가 되도록 비율을 재조정
    if total_variable > 0:
        # 스케일 인자 계산(소수점 둘째자리까지 반올림)
        scale_factor = (100 - total_fixed) / total_variable
        # 가변 비율에 대해 스케일 인자를 곱하여 재조정(소수점 둘째자리까지 반올림)
        group.loc[variable_mask, '점유율'] = (group.loc[variable_mask, '점유율'] * scale_factor).round(2)
        
    return group

# 법정동코드로 그룹화하여 점유율 값을 조정
data_shp_adj = data_shp.groupby('법정동코드', group_keys=False).apply(adjust_유역면적비율s)

# 각 법정동코드 그룹별로 재조정 후의 합계를 확인(확인용)
group_sum = data_shp_adj.groupby('법정동코드')['점유율'].sum().reset_index()

data_shp_adj


C:\Users\TG\AppData\Local\Temp\ipykernel_51752\336400320.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_shp_adj = data_shp.groupby('법정동코드', group_keys=False).apply(adjust_유역면적비율s)


,법정동코드,동리코드,시도,시군,읍면동,리,동리면적,단위유역,유역면적,유역면적비율,점유율,동리개수
0,5111010100,춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0,100.0,1
1,5111010200,춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0,100.0,1
2,5111010300,춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0,100.0,1
3,5111010400,춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0,100.0,1
4,5111010500,춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,None,2.518901e+06,100.0,100.0,1
1903,5183035039,양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,None,9.178029e+05,100.0,100.0,1
1904,5183035040,양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,None,4.658669e+05,100.0,100.0,1
1905,5183035041,양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,None,1.361706e+06,100.0,100.0,1


In [29]:
# 점유율이 0인 행을 제거
data_shp_adj = data_shp_adj[data_shp_adj['점유율'] != 0]
# 점유율이 0인 행을 제거한 후, 다시 동리개수 열 계산
data_shp_adj['동리개수'] = data_shp_adj.groupby(['동리코드'])['동리코드'].transform('count')

# 단위유역 열의 값이 결측인 경우 값을 "기타"로 설정
data_shp_adj['단위유역'] = data_shp_adj['단위유역'].fillna('기타')

# '법정동코드' 열을 기준으로 오름차순 정렬
data_shp_adj = data_shp_adj.sort_values(by='법정동코드')

data_shp_adj

C:\Users\TG\AppData\Local\Temp\ipykernel_51752\2216915112.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shp_adj['동리개수'] = data_shp_adj.groupby(['동리코드'])['동리코드'].transform('count')
C:\Users\TG\AppData\Local\Temp\ipykernel_51752\2216915112.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shp_adj['단위유역'] = data_shp_adj['단위유역'].fillna('기타')


,법정동코드,동리코드,시도,시군,읍면동,리,동리면적,단위유역,유역면적,유역면적비율,점유율,동리개수
0,5111010100,춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0,100.0,1
1,5111010200,춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0,100.0,1
2,5111010300,춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0,100.0,1
3,5111010400,춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0,100.0,1
4,5111010500,춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,기타,2.518901e+06,100.0,100.0,1
1903,5183035039,양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,기타,9.178029e+05,100.0,100.0,1
1904,5183035040,양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,기타,4.658669e+05,100.0,100.0,1
1905,5183035041,양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,기타,1.361706e+06,100.0,100.0,1


In [30]:
# 엑셀 파일로 내보내기
data_shp_adj.to_excel('E:/Coding/TMDL/전국오염원조사/동리별_유역현황.xlsx', index=False)
